In [ ]:
version               = ""
input_dir             = "./changes"
outdir                = "./figures"
figure_name           = "fig_SOD_BoxTS.X_f3_h"
panel                 = "h"
title                 = ""
yaxis_title           = "%% change in space averages "

#
data_versions_tag     = "20200918"
excluded_models       = []
variables             = [("pr","mean"),("mrro","mean"), ("pr","std"), ("mrro","std")]
option                = "mean"
combined_seasons      = [ "tropics_JJA", "extra_summer" ]

#
colors                = {"tropics_JJA"     : {"pr":"indianred1","mrro":"cornflowerblue"},
                         "extra_summer"    : {"pr":"firebrick3","mrro":"blue3"},
                         "land_annual"     : { "pr":"firebrick3","mrro":"blue3","prw":"grey","tas":"grey"},
                         "tropics_annual"  : { "pr":"firebrick3","mrro":"blue3","prw":"grey","tas":"grey"}}
thicknesses           = {"tropics_JJA" :10.0,"extra_summer" : 4.0 , "land_annual":10.0, "tropics_annual":10.0}
xyMarkLineModes       = {"mean":"MarkLines","std":"MarkLines"}
xyDashPatterns        = {"mean":0,"std":1 }
xyMarkerSizes         = {"pr_mean" : 0.015,"mrro_mean" : 0.015,"pr_std" : 0.01,"mrro_std": 0.01, "prw_mean": 0.01,"tas_mean":0.015, "tas_std":0.01}
nice                  = {"pr":"precipitation","prw":"precipitable water ","mrro":"runoff",
                        "tropics_JJA":"Tropical land JJA",
                        "extra_summer" : "Extra-tropical summer ~C~(NH:JJA, SH:DJF)",
                        "land_annual" : "Land annual", "tropics_annual" : "Tropics annual", 
                        "mean":"annual mean", "std" : "inter-annual variability",
                        }
only_warmer_CI        = False # Should we show Confidence Interval bars only at warm end
show_variability_CI   = True  # Should we show Confidence Interval bars also for variability 
show_mean_CI          = True  # Should we show Confidence Interval bars also for means 
show_tas_CI           = False # Should we show Confidence Interval bars for GSAT
xy_ranges             = (0.,6.5,-15.,60.)
#
max_warming_level     = 5.
min_models_nb         = 20

do_test               = True

In [ ]:
if do_test :
    version="_test"
    variables             = [("mrro","mean")]
    combined_seasons      =  [ "tropics_annual" ]


In [ ]:
# Parameters for TS figure
do_test = False
xy_ranges = [1.5, 5.8, -15.0, 50.0]
combined_seasons = ["tropics_annual"]
dummy = "dummy"
variables = [["pr", "mean"], ["mrro", "mean"], ["pr", "std"], ["mrro", "std"], ["prw", "mean"]]
show_variability_CI = True
yaxis_title = "% change over tropical land"
figure_name = "fig_SOD_BoxTS.X_f3_h_tropics_3vars_tas"
data_versions_tag = "20200918"
title = "Hydrological variables change over tropical land"
input_dir = "/data/ssenesi/prod/fig_SOD_BoxTS.X_f3_h_wl/changes"
excluded_models = ["CAMS-CSM1-0"]
version = "tropics_ann_ssp5"
only_warmer_CI = True
use_cached_proj_fields = False
show_mean_CI = True
show_tas_CI = False 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import numpy.ma as ma
import Ngl
import os
import json
from IPython.display import Image

In [ ]:
def ensemble_stat_series(stats,variable,stat,season,option,min_models_nb=min_models_nb,max_warming_level=max_warming_level) :
    """ Compute mean or quantile on an ensemble """
    #
    global excluded_models
    
    series=stats[variable][stat][season]["ens"]
    periods=series.keys()
    periods.sort()
    ret=[]
    for p in periods :
        if compute_option != "parametric" and float(p)> max_warming_level :
            ret.append(missing_value)
            continue
        ens=series[p]
        # Remove ensemble entries which model_scenario key includes an excluded model
        for excl in excluded_models :
            for m in ens.keys() :
                if excl in m :
                    ens.pop(m)
        if len(ens) < min_models_nb:
            ret.append(missing_value)
            continue
        if option == "x":
            ret.append(np.float64(p))  ############################################################### a revoir pour le cas parametric
            continue
        ####
        ens=ens.values()
        if option == "mean":
            ret.append(np.mean(ens))
        elif option == "median":
            ret.append(np.median(ens))
        #
        elif option in ["min","max"] :
            l=[ value for value in ens ]
            l.sort()
            if option=="min"     : ret.append(np.float64(l[0]))
            if option=="max"     : ret.append(np.float64(l[-1]))
        #
        elif option in ["nq5","nq95" ] : #percentiles with gaussian hypothesis
            mean=np.mean(ens)
            std1=np.std(ens,ddof=1)
            if option == "nq5"   : ret.append(mean - 1.645*std1)
            if option == "nq95"  : ret.append(mean + 1.645*std1)
        #
        else:
            raise ValueError("Time to use some library for stats (%s) !"%option)
    #print "time series for %s"%option, ret
    return(ret)

In [ ]:
def season_plot(wks,stats,season,variables,option,errors,only_warmer=only_warmer_CI,
                show_variability_CI=show_variability_CI,show_tas_CI=show_tas_CI,
                show_mean_CI=show_mean_CI,
                rank=1,key="",xy_ranges=(0.,6.5,-15.,60.),title=""):
    """
    rank : allows to decide for a shift of the legend
    key : text for the legend
    """
    #GTAS_shift=0.89 # Shift for GTAS between 1995-2014 and 1850-1900, based on AR6 WL Box table
    #GTAS_shift=0.   # No shift if GTAS computed for 1850-1900
    #
    # Gathering time series of ensemble mean of time statistics for required variables
    means=[]  # their mean value
    x=[] # their x-axis, which is a warming level if compute_option != parametric
    for variable,stat in variables :
        #d=stats[variable][stat][season][option]
        #periods=sorted(d) 
        #means.append([ d[p] for p in periods ])
        means.append(ensemble_stat_series(stats,variable,stat,season,option))
        x.append    (ensemble_stat_series(stats,variable,stat,season,"x"   ))
    #
    if len(means)==0 : 
        raise ValueError("No matching variable in stats "+repr(stats.keys()))
    #print "means=",means
    if len(means)>0 :
        y=np.array(means,np.float32)
        y=ma.masked_values(y,missing_value)
        if compute_option == "parametric" : 
            periods  = stats["tas"]["mean"]["globe_year"]["mean"].keys()
            periods.sort()
            GTAS  = [ stats["tas"]["mean"]["globe_year"]["mean"][p] for p in periods ]
        else:
            GTAS=np.array(x)

        #-- set resources
        res                  =  Ngl.Resources()       #-- generate an res object for plot
        res.nglDraw    = False                        # don't draw
        res.nglFrame   = False                        # don't advance frame
        res.gsnMaximize = False
        res.gsnPaperOrientation="landscape"
        res.tiMainString = title
        res.tiMainFontHeightF = 0.02
        #
        res.trXMinF,res.trXMaxF,res.trYMinF,res.trYMaxF = xy_ranges
        #
        res.tmYMajorGrid  = True
        dpat = Ngl.new_dash_pattern(wks,"_____$_____$_____$_____$_____$_____$_____$_____$_____$_____")
        res.tmYMajorGridLineDashPattern =  dpat #15
        
        #res.tmYLMode = "Explicit"
        res.tmYLValues = [-10.,0.,10.,20.,30.,40.,50.]
        #res.tmYLMode = "Automatic"
        res.tmYLMode = "Manual"
        res.tmYLTickStartF  = -20.
        res.tmYLDataBottomF = -20.
        res.tmYLDataTopF    =  40.
        res.tmYLTickSpacingF=  20
        #
        res.tmXBMode = "Explicit"
        res.tmXBValues = [ 2,3,4,5]
        res.tmXBLabels = [ "2","3","4","5"] 
        res.tmXBTickStartF  =   0.
        res.tmXBTickSpacingF=   1.
        

        res.tmXBDataLeftF   =  1.4
        res.tmXBDataRightF  =  5.4
        #
        res.tiYAxisString    = yaxis_title + " (multi-model mean)"
        res.tiYAxisFontHeightF = 0.016
        res.tiXAxisString    = "GSAT change since pre-industrial"   #-- x-axis title
        res.tiXAxisFontHeightF = 0.016
        
        #-- xy-plot resources
        res.xyLineColors     = [colors[season][v] for v,s in variables] #-- set 3 different colors for lines
        res.xyLineThicknessF = [thicknesses[season]]               #-- line thickness for all
        res.xyDashPatterns   = [ xyDashPatterns [s] for v,s in variables]
        #
        res.xyMarkLineModes  = [ xyMarkLineModes[s] for v,s in variables]
        res.xyMarker         = 16
        res.xyMarkerSizes    = [ xyMarkerSizes.get("%s_%s"%(v,s),0.01) for v,s in variables]
        res.xyMarkerColors   = [colors[season][v] for v,s in variables] 

        #-- legend resources
        res.lgAutoManage     = False
        res.lgLabelJust      = "CenterLeft"
        res.lgBoxBackground  = 0
        res.pmLegendDisplayMode = "Always" 
        res.lgLabelFontAspectF = 1.5
        #res.lgLabelConstantSpacingF =0.0
        if rank == 1 :
            res.xyExplicitLegendLabels = ["  "+ nice[v]+" "+nice[s] for v,s in variables ]  #-- set explicit legend labels
            res.pmLegendParallelPosF =  -0.45     
        else :
            res.xyExplicitLegendLabels = [ "" for v,s in variables ]  #-- no legend labels
            res.pmLegendParallelPosF =  0.15       
        #
        res.pmLegendOrthogonalPosF =  0.4         #-- move the legend upwards
        res.pmLegendZone     =  0                     #-- legend zone: 0 = topLeft; 6 = topRight
        #res.pmLegendOrthogonalPosF =  0.5           #-- move the legend upwards
        res.lgJustification  = "TopLeft"          #-- legend justification
        res.pmLegendWidthF   =  0.15                  #-- change width
        res.pmLegendHeightF  =  0.1
        res.lgLabelFontHeightF = 0.015
        res.pmLegendSide     = "Top"                  #-- Change location
        res.lgPerimOn        =  False                 #-- turn off the perimeter
        #
        #res.xyLabelMode = "Custom"
        #res.xyLineLabelFontColors = res.xyLineColors
        #res.xyLineLabelFont  = "times-roman"
        #res.xyLineLabelFontQuality ="High"
        res.xyLineLabelFontColors = [colors[season][v] for v,s in variables]
        if key is None :
            res.pmLegendOrthogonalPosF =  0.47 #1.6         #-- move the legend upwards
        #-- creates the plot
        plot=Ngl.xy(wks,GTAS,y,res)
        #
        # Plotting model numbers for each (or the first) variable
        #
        count=0
        deltay=3
        processed=[]
        print "variables=",variables
        for var,stat in variables :
            if var in processed : continue
            #first_var,_=variables[0]
            processed.append(var)
            print "processing",var
            models_nb=stats[var]["models_number"]
            if compute_option != "parametric" : 
                resn=  Ngl.Resources() 
                resn.txFontHeightF = 0.025
                resn.txJust ="TopRight"
                resn.txFontColor = colors[season][var]
                y_model_numbers = res.trYMinF + 8 + count*deltay
                print "y_model_numbers=",y_model_numbers
                Ngl.add_text(wks, plot, "models #", 2. - 0.3, y_model_numbers, resn)
                resn.txJust ="TopCenter"
                for x in models_nb :
                    mn="%d"%models_nb[x]
                    Ngl.add_text(wks, plot, mn, x, y_model_numbers, resn)
                count += 1
        #
        # Key label
        ############
        #
        if key is not None :
            rest=  Ngl.Resources() 
            rest.txFontHeightF = 0.02
            rest.txJust ="TopCenter"
            rest.txFuncCode ="~"
            if rank==1:
                xt=1.2
            elif rank== 2 :
                xt=5.2
            Ngl.add_text(wks, plot, key, xt, res.trYMaxF - 3., rest)
        #
        #
        # Handling error bars
        #####################
        #
        def dic2list_old(stat):
            alt_season=season
            if v=="tas" :
                alt_season="globe_year"
            dic=stats[v][s][alt_season][stat]
            periods=dic.keys()
            periods.sort()
            l=[ dic[p] for p in periods ]
            return np.array(l,np.float32)
        def dic2list(stat):
            l=ensemble_stat_series(stats,v,s,season,stat)
            return np.array(l,np.float32)
        #
        CI_vars=variables
        if show_mean_CI is False :
            CI_vars=[ (var,stat) for (var,stat) in CI_vars if stat != "mean" ]
        if show_variability_CI is False :
            CI_vars=[ (var,stat) for (var,stat) in CI_vars if stat != "std" ]
            # CI_vars=[ (var,stat) for (var,stat) in CI_vars if ( var != "prw" or stat != "std" ) ]
        if show_tas_CI and compute_option == "parametric" : 
            CI_vars  = CI_vars + [("tas","mean")]
        #
        index=-1
        delta_scale=2.
        for v,s in CI_vars :
            print v,s
            index+=1
            if errors in ["second","butlast"] :
                ups =dic2list("butlast")
                lows =dic2list("second")
            elif errors in ["nq5", "nq95"] :
                ups =dic2list("nq95")
                lows =dic2list("nq5")
            elif errors in ["max", "min"] :
                ups =dic2list("max")
                lows =dic2list("min")
            elif errors == "means"  and "wmean" in stats[v][season] :
                ups =dic2list("wmean")
                lows =dic2list("iwmean")
            else : raise ValueError("errors option '%s' not (yet?) available"%errors)
            centers=dic2list("mean")
            polyres                   =  Ngl.Resources()
            if s=="std" :
                polyres.gsMarkerIndex     = 1        
            else :
                #polyres.gsMarkerIndex     = 2        
                polyres.gsMarkerIndex     = 1        
            polyres.gsMarkerSizeF     = .06     
            polyres.gsMarkerColor     = colors[season][v]
            polyres.gsLineColor       = colors[season][v]
            polyres.gsLineThicknessF  = 10. #2.
            polyres.gsLineDashPattern = xyDashPatterns[s]
            #polyres.gsLineDashPattern = xyDashPatterns["mean"]
            #
            # x-wise shift, dependent on variable+stat
            if v=="pr" : 
                if s != "std":
                    delta=0.
                else :
                    delta=0.14
            elif v=="mrro"  :
                if s!="std"  :
                    delta=0.04
                else :
                    delta=0.18
            else : delta=0.
            #
            x=ensemble_stat_series(stats,v,s,season,"x"   )
            all_error_bars=zip(x,centers,ups,lows)                     ####################################### etait GTAS
            #extremes=[ all_error_bars[0],all_error_bars[-1]]
            first=True
            if only_warmer :
                extremes=[ all_error_bars[-1]]
                first=False
            extremes=all_error_bars
            for t,center,up,low in extremes :
                if t > 1.e+10 : continue
                if extremes != all_error_bars :
                    if first : t0=xy_ranges[0]+0.1
                    else : t0=xy_ranges[1]-1.
                    if rank==1 : t0=t0+0.5
                else :
                    t0=t
                if v != "tas" :
                    Ngl.add_polymarker(wks,plot,t0+delta*delta_scale,up,polyres)
                    Ngl.add_polymarker(wks,plot,t0+delta*delta_scale,center,polyres)
                    Ngl.add_polymarker(wks,plot,t0+delta*delta_scale,low,polyres)
                    Ngl.add_polyline  (wks,plot,[t0+delta*delta_scale,t0+delta*delta_scale],[up,low],polyres)
                    if v=="prw" : 
                        x_prw=t0+delta*delta_scale
                        y_prw=center
                elif "prw" in [ var for var,stat in CI_vars ]  :
                    Ngl.add_polymarker(wks,plot,x_prw+low-center,y_prw,polyres)
                    Ngl.add_polymarker(wks,plot,x_prw,           y_prw,polyres)
                    Ngl.add_polymarker(wks,plot,x_prw+up-center, y_prw,polyres)
                    Ngl.add_polyline  (wks,plot,[x_prw+up-center,x_prw+low-center],[y_prw,y_prw],polyres)
                first=False
                #
                if index==0 and rank==1 : #and season  in [ "extra_summer", "land_annual" ]:
                    rest=  Ngl.Resources() 
                    rest.txFontHeightF = 0.015
                    rest.txJust ="Center"
                    #Ngl.add_text(wks, plot, "[5%-95%] range" , t0-0.7, res.trYMinF+5., rest)
        return plot

In [ ]:
def combined_figure(stats,outname,variables,option,errors):
    #
    wks_type        = "png"                       #-- output type of workstation
    wks             =  Ngl.open_wks(wks_type,outname)
    #
    # If actually combining seasons, the key must show each season name 
    # otherwise, we assume that the title includes it 
    if len(combined_seasons)==1 : 
        key=None
    else :
        key=nice[combined_seasons[0]]
    plot1=season_plot(wks,stats,combined_seasons[0],variables,option,
                      errors,only_warmer_CI,show_variability_CI,rank=1,key=key,
                      xy_ranges=xy_ranges,title=title)
    #return "debug"
    if len(combined_seasons) == 2 : 
        key=nice[combined_seasons[1]]
        plot2=season_plot(wks,stats,combined_seasons[1], variables,option,
                          errors,only_warmer_CI,show_variability_CI,rank=2, key=key,
                          xy_ranges=xy_ranges)
        Ngl.overlay(plot1,plot2)
    Ngl.draw(plot1)
    #
    Ngl.frame(wks)         # flush graphics to the wks
    Ngl.delete_wks(wks)    #-- this need to be done to cope with opened wks max number
    return outname+"."+wks_type


In [ ]:
fn="%s/stats_CMIP6_%s%s.json"%(input_dir,data_versions_tag,version)
print fn
with open (fn,"r") as f :
    stats=json.load(f)
print stats.keys()
ref_period=stats["ref_period"]
compute_option=stats.get("option","parametric").encode('ascii')
missing_value=1.e+20
#stats=stats['ssp585']

In [ ]:
#show_tas_CI = True
only_warmer_CI = False
xy_ranges = [1.5, 5.8, -20.0, 60.0]
variables = [["pr", "mean"],  ["prw", "mean"], ['mrro','mean']]
variables = [("pr","std"), ("mrro","std"), ("prw", "mean")]

max_warming_level     = 20.
min_models_nb         = 10


#xy_ranges = [0., 5.8, -10.0, 50.0]

In [ ]:
filen="%s/%s_%s_%s%s"%(outdir,figure_name,compute_option,data_versions_tag,version)
! mkdir -p {outdir}

figfile=combined_figure(stats,filen,variables,option, errors="nq95")

print "Image available as %s"%figfile
Image(figfile)

In [ ]:
all_metadata=""
for variable in stats.keys():
    if variable not  in [ var for var,stat in variables ] : continue
    print type(stats[variable])
    metadata=stats[variable]["metadata"]
    for scenario in metadata :
        for model in metadata[scenario] :
            if model not in excluded_models :
                string = metadata[scenario][model]
                string.replace("\n"," %s\n"%panel)
                all_metadata += string
with open("%s_md"%(filen),"w") as f: f.write(all_metadata)